<a href="https://colab.research.google.com/github/codebjjung/ML-DL/blob/main/CNN_seq_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# libraries

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os

In [ ]:
# set hyper parameters

learning_rate = 0.001
training_epochs = 15
batch_size = 100

cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'minst_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

In [ ]:
# make a data pipelining

mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis = -1)
test_images = np.expand_dims(test_images, axis = -1)

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).shuffle(buffer_size=100000).batch(batch_size)

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
# model function #

def create_model():
  model = keras.Sequential()
  # Conv
  model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', input_shape=(28,28,1)))

  # Max pooling
  model.add(keras.layers.MaxPool2D(padding='SAME'))

  # Conv
  model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))

  # Max pooling
  model.add(keras.layers.MaxPool2D(padding='SAME'))

  # Conv
  model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))

  # Max pooling
  model.add(keras.layers.MaxPool2D(padding='SAME'))

  # Fully connected layer
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(256,activation=tf.nn.relu))
  model.add(keras.layers.Dropout(0.4))
  model.add(keras.layers.Dense(10))
  return model

In [ ]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 128)         0

In [ ]:
def loss_fn(model, images, labels):
  logits = model(images, training=True)
  ## bugfix (logits_v2 -> logits)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
  return loss

In [ ]:
# calculating gradient

def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [ ]:
def evaluate(model, images, labels):
  logits = model(images, training=False)
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return accuracy

## adam optimizer debug (tf.train.AdamOptimizer -> tf.keras.optimizers.Adam)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

checkpoint = tf.train.Checkpoint(cnn=model)

In [ ]:
for epoch in range(training_epochs):
  avg_loss = 0.
  avg_train_acc = 0.
  avg_test_acc = 0.
  train_step = 0
  test_step = 0

  for images, labels in train_dataset:
    grads = grad(model, images, labels)
    optimizer.apply_gradients(zip(grads, model.variables))
    loss = loss_fn(model, images, labels)
    acc = evaluate(model, images, labels)
    avg_loss = avg_loss + loss
    avg_train_acc = avg_train_acc + acc
    train_step += 1

  avg_loss = avg_loss / train_step
  avg_train_acc = avg_train_acc / train_step

  for images, labels in test_dataset:
    acc = evaluate(model, images, labels)
    avg_test_acc = avg_test_acc + acc
    test_step += 1

  avg_test_acc = avg_test_acc / test_step

  print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 'train accuracy =', '{:.4f}'.format(avg_train_acc), 'test accuracy = ', '{:.4f}'.format(avg_test_acc))
  checkpoint.save(file_prefix=checkpoint_prefix)

Epoch: 1 loss = 0.19115287 train accuracy = 0.9543 test accuracy =  0.9844
Epoch: 2 loss = 0.05215061 train accuracy = 0.9890 test accuracy =  0.9902
Epoch: 3 loss = 0.03524313 train accuracy = 0.9927 test accuracy =  0.9914
Epoch: 4 loss = 0.02582709 train accuracy = 0.9948 test accuracy =  0.9929
Epoch: 5 loss = 0.02180528 train accuracy = 0.9961 test accuracy =  0.9906
Epoch: 6 loss = 0.01839465 train accuracy = 0.9969 test accuracy =  0.9930
Epoch: 7 loss = 0.01488997 train accuracy = 0.9975 test accuracy =  0.9927
Epoch: 8 loss = 0.01242652 train accuracy = 0.9980 test accuracy =  0.9941
Epoch: 9 loss = 0.01014663 train accuracy = 0.9987 test accuracy =  0.9932
Epoch: 10 loss = 0.00903663 train accuracy = 0.9989 test accuracy =  0.9915
Epoch: 11 loss = 0.00781323 train accuracy = 0.9991 test accuracy =  0.9935
Epoch: 12 loss = 0.00724865 train accuracy = 0.9993 test accuracy =  0.9913
Epoch: 13 loss = 0.00679678 train accuracy = 0.9992 test accuracy =  0.9930
Epoch: 14 loss = 0.00